Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML: https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Beautiful Soup

**Beautiful Soup** es una librería **Python** que permite extraer información de contenido en formato **HTML o XML**. Para usarla, es necesario especificar un **parser**, que es responsable de transformar un documento HTML o XML en un árbol complejo de objetos Python. Esto permite, por ejemplo, que podamos interactuar con los elementos de una página web como si estuviésemos utilizando las herramientas del desarrollador de un navegador.

A la hora de extraer información de una web, uno de los parsers más utilizado es el parser HTML de **lxml**. Precisamente, será el que utilicemos en este tutorial.

**Será necesario instalar las siguientes librerías** (si no las tienes ya):

        pip3 install beautifulsoup4 requests pandas

        pip3 install beautifulsoup4 

        pip3 install requests

        pip3 install pandas 

###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

Algunos ejemplos de Web Scraping utilizando Beautiful Soup:

https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/

http://omz-software.com/pythonista/docs/ios/beautifulsoup_guide.html

https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [5]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html
import numpy as np
import lxml

## Caso 1: Scraping de un catálogo: Labirratorium

In [6]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [7]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página. 958 cervezas en total

Hacemos la consulta (request) y creamos la SOPA inicial:

In [8]:
# 1. hacemos el request de la pagina. Es decir nos estamos conectando a la pagina
# 2. en la sopa se parsear es decir para como traducir el html que tenemos o acceder a el. Es la manera de traducir 

In [9]:
r = requests.get(URL)
soup  = BeautifulSoup(r.text,'lxml') #estamos creando la sopa



In [10]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_ = 'product-image') # le vamos a aplicar un metodo propio de beautiful soup para decirle que nos encuentre todo lo que queramos de untipo. Con este grid estoy guardando todos los elementos cervezas grid
len(cervezas_grid)


12

#### 3.  necesitamos acceder a cada una de las cervezas del grid:

In [11]:

 # como son varias cervezas o varias pantnallas hay que acceder a cada una de las urls y hacer un request
lista_URL = [] #como yo necesito las 12 me hago una lista vacia para traerlas haciendo un append
for cerveza in cervezas_grid:
     URL_cerveza = cerveza.find('a')['href'] #se pone a por que en la pagina web la url esta contenida dentro de ese 'a' y 'a' es un tag o un tipo de contenedor. Como del a solo quiero href, accedo como en una lista al href
     lista_URL.append(URL_cerveza)
lista_URL

['https://www.labirratorium.com/es/lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/alemania/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/inicio/199-orval.html',
 'https://www.labirratorium.com/es/alemania/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/165-laugar-epa.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html']

#### 4. Hacemos un nuevo request para la primera cerveza: 


In [12]:
r = requests.get(lista_URL[0])
#una vez que tenemos el request hecho hay que crear una sopa especifica para la nueva cerveza
soup_cerveza = BeautifulSoup(r.text, 'lxml')

#### 5. Vamos a empezar a traernos ya la informacion que nos interesa para nuestro df

In [13]:
# Nombre
nombre = soup_cerveza.find(class_ = 'h1 product-detail-name').text
print(nombre)


Boon Oude Kriek 37,5cl


In [14]:
# Precio
precio = soup_cerveza.find(class_ = 'current-price').find('span')['content']#aqui vamos a tirar del hilo por que vimos que en content estaba el valor que nos interesaba 
precio


'7.15'

In [15]:
# Descripcion corta

descrp_corta = soup_cerveza.find(class_ = 'description-short').find('p').text  #aqui como lo que queremos es el text le ponemos.text 
descrp_corta


'Lambic / Kriek'

In [16]:
# Descripción larga
descrp_larga = soup_cerveza.find(class_ = 'product-description').find('p').text
descrp_larga


'Cerveza de fermentación espontánea (Lambic) de 6.5% ABV sin filtrar ni pasteurizar de estilo Kriek, elaborada con cerezas naturales.'

In [17]:
# Imagen
imagen = soup_cerveza.find(class_ = 'js-qv-product-cover img-fluid')['src']
imagen


'https://www.labirratorium.com/19351-large_default/boon-kriek-2013.jpg'

In [18]:
 # Brand
 marca = soup_cerveza.find(class_ = 'img img-thumbnail manufacturer-logo')['alt']
 marca 


'Brouwerij F. Boon'

In [19]:
# Código de barras


In [20]:
# Features
features = soup_cerveza.find(class_ = 'data-sheet')
features_dic = {}#se convierte a diccionario dt es la clave y el dd es el valor
features_dt = features.find_all('dt')
features_dd = features.find_all('dd')

for feature, value in zip(features_dt, features_dd): #esto para unir una pareja. Y se va construyendo esta clave valor dentro del diccionario
    features_dic[feature.text] = value.text #aqui le digo que el feature sean las clavez y el valor como text

features_dic


{'Estilo': 'KRIEK',
 'Origen': 'Bélgica',
 '% Alc.': '6.5\nALTO (6-9%)',
 'Otros ingredientes': 'Cerezas Naturales',
 'Volumen (cl)': '37.5 Cl',
 'Tipo Fermentación': 'Lambic (Fermentación espontánea o salvaje)',
 'Maltas': 'Cebada y Trigo',
 'IBU': '0-25 Amargor bajo',
 'Color': 'Rojiza',
 'Envase': 'Botella'}

In [21]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
id_cerv = 'lbt_'+str() #esto es algo que ponemos nosotros. aqui le puso lbt por que asi se llama la pagina o mas o menos asi


Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [22]:
# Agregamos a una lista


In [23]:
## CODIGO QUE PASO BORJA 
# Agregamos a una lista
pages = np.arange(1,81)
count = 1
lista_cervezas = []

for page in pages:

    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    cervezas_grid = soup.find_all(class_='product-image')
    
    count_beer = 1 #esto solo va de 1 a 12 en cada pagina

    for cerveza in cervezas_grid:
        print('Cerveza {} de {}, pag {}/{}'.format(
            count_beer, len(cervezas_grid), page, len(pages))) # va a hacer un print de cada cerveza de cada pagina. Este va de 1 a 998
        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        soup_cerveza = BeautifulSoup(r.text, 'lxml')

        id_cerv = 'lbt_' + str(count)

        name = soup_cerveza.find(class_='h1 product-detail-name').text   

        price = soup_cerveza.find(class_='current-price').find('span')['content']

        try:
            descr_short = soup_cerveza.find(class_='description-short').find('p').text
        except:
            descr_short = None

        image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']

        try:
            brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
        except:
            brand = None

        features_dicc = {}
        features = soup_cerveza.find(class_ = 'data-sheet')
        try:
            features_dt = features.find_all('dt')
            features_dd = features.find_all('dd')
            for feature, value in zip(features_dt, features_dd):
                #print(feature.text, value.text)
                features_dicc[feature.text] = value.text
        except:
            features_dicc = {}

        lista_cervezas.append([
    ('id', id_cerv),
    ('name', name), 
    ('price', price),
    ('descr_short', descr_short),
    ('image', image),
    ('brand', brand),
    ('features', features_dicc)
    ]) 
    count_beer += 1 
count +=1

 

 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 19/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 20/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 21/80
Cerveza 1 de 12, pag 22/80
Cerveza 1 de 12, pag 22/80
Cerveza 

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

In [26]:
df = pd.DataFrame([[x[0][1], 
                    x[1][1], 
                    x[2][1],
                    x[3][1],
                    x[4][1],
                    x[5][1],
                    x[6][1]] for x in lista_cervezas],
                columns = ['id', 
                            'name',
                            'price',
                            'descr_short',
                            'imagen',
                            'brand',
                            'features'])
df.to_csv('df_labirratorium.csv', sep=';')
df 

,id,name,price,descr_short,imagen,brand,features
0,lbt_1,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,https://www.labirratorium.com/19351-large_defa...,Brouwerij F. Boon,"{'Estilo': 'KRIEK', 'Origen': 'Bélgica', '% Al..."
1,lbt_1,Störtebeker Schwarz-Bier,2.4,Cerveza negra de baja fermentación,https://www.labirratorium.com/488-large_defaul...,Störtebeker,"{'Estilo': 'SCHWARZBIER', 'Origen': 'Alemania'..."
2,lbt_1,Orval,2.8,Belgian Pale Ale,https://www.labirratorium.com/385-large_defaul...,Orval,"{'Estilo': 'BELGIAN PALE ALE', 'Origen': 'Bélg..."
3,lbt_1,Augustiner Lagerbier Hell,2.5,Lager / Helles,https://www.labirratorium.com/367-large_defaul...,Augustiner,"{'Estilo': 'MUNICH HELLES', 'Origen': 'Alemani..."
4,lbt_1,Schneider Aventinus Weizen-Eisbock,2.7,"Cerveza de color medio oscuro, aromas alcohóli...",https://www.labirratorium.com/366-large_defaul...,Schneider,"{'Origen': 'Alemania', '% Alc.': '12', 'Volume..."
...,...,...,...,...,...,...,...
953,lbt_1,St. Peters Without GOLD Sin Alcohol 0.0,2.8,Sin Alcohol / Blonde Ale,https://www.labirratorium.com/12551-large_defa...,St Peters,"{'Origen': 'Inglaterra', '% Alc.': '0.0% SIN A..."
954,lbt_1,Pohjala Prenzlauer 0,3.6,Sin Alcohol / Fruit Berliner Weisse,https://www.labirratorium.com/22380-large_defa...,Pohjala (Põhjala),"{'Estilo': 'SIN ALCOHOL', 'Origen': 'Estonia',..."
955,lbt_1,Lervig No Worries Lemon,2.9,Sin Alcohol / Fruit IPA,https://www.labirratorium.com/21954-large_defa...,Lervig,"{'Estilo': 'SIN ALCOHOL', 'Origen': 'Noruega',..."
956,lbt_1,Mikkeller Baghaven Oud Sasughaven Blend 1,21.9,Lambic / Saison,https://www.labirratorium.com/22353-large_defa...,Mikkeller Baghaven,"{'Estilo': 'GUEUZE', 'Origen': 'Dinamarca / Bé..."


### FBI: Top ten criminals

#### Queremos guardar las imágenes de cada fugitivo y que el nombre de cada archivo sea el nombre del fugitivo:

In [27]:
fbi_url = 'https://www.fbi.gov/wanted/topten'



In [25]:
r = requests.get(fbi_url)
soup  = BeautifulSoup(r.text,'lxml') 

In [ ]:
# Guardamos lista de criminales
cervezas_grid = soup.find_all(class_ = '')